#Samer Baslan
#CMPE-297: Assignment 3 Part 3
#BERT Masked Fine Tuning

Resources: <br />
https://github.com/huggingface/notebooks/blob/master/examples/multiple_choice.ipynb


##Installs

In [1]:
!pip install git+https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/datasets.git
!pip install gradio

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-lycbfsre
  Running command git clone -q https://github.com/huggingface/transformers.git /tmp/pip-req-build-lycbfsre
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 15.0 MB/s 
     |████████████████████████████████| 56 kB 4.0 MB/s 
     |████████████████████████████████| 3.3 MB 55.0 MB/s 
     |████████████████████████████████| 596 kB 57.9 MB/s 
  Created wheel for transformers: filename=transformers-4.13.0.dev0-py3-none-any.whl size=3074080 sha256=9d002fd19bcc093719fcb81f74854be157a4d8f776a4c803a1fc619535524316
  Stored in directory: /tmp/pip-ephem-wheel-cache-ro96qdcf/wheels/90/a5/44/6bcd83827c8a60628c5ad602f429cd5076bcce5f2a90054947
Successfully built transformers
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successf

##Imports

In [5]:
import transformers
from transformers import AutoTokenizer, AutoModelForMultipleChoice, \
TrainingArguments, Trainer
print(transformers.__version__)

from datasets import load_dataset, load_metric
from datasets import ClassLabel
import random
import numpy as np
import pandas as pd
from IPython.display import display, HTML

from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch


4.13.0.dev0


##Setup

In [6]:
model_checkpoint = "bert-base-uncased"
batch_size = 16

In [7]:
datasets = load_dataset("swag", "regular")

Downloading:   0%|          | 0.00/2.35k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/6.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.24M [00:00<?, ?B/s]

...

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset swag downloaded and prepared to /root/.cache/huggingface/datasets/swag/regular/0.0.0/9640de08cdba6a1469ed3834fcab4b8ad8e38caf5d1ba5e7436d8b1fd067ad4c. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
datasets

DatasetDict({
    train: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 73546
    })
    validation: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20006
    })
    test: Dataset({
        features: ['video-id', 'fold-ind', 'startphrase', 'sent1', 'sent2', 'gold-source', 'ending0', 'ending1', 'ending2', 'ending3', 'label'],
        num_rows: 20005
    })
})

In [9]:
datasets["train"][0]

{'ending0': 'passes by walking down the street playing their instruments.',
 'ending1': 'has heard approaching them.',
 'ending2': "arrives and they're outside dancing and asleep.",
 'ending3': 'turns the lead singer watches the performance.',
 'fold-ind': '3416',
 'gold-source': 'gold',
 'label': 0,
 'sent1': 'Members of the procession walk down the street holding small horn brass instruments.',
 'sent2': 'A drum line',
 'startphrase': 'Members of the procession walk down the street holding small horn brass instruments. A drum line',
 'video-id': 'anetv_jkn6uvmqwh4'}

In [10]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(datasets["train"])

,video-id,fold-ind,startphrase,sent1,sent2,gold-source,ending0,ending1,ending2,ending3,label
0,lsmdc3042_KARATE_KID-19490,8605,They weave through the crowd. Stylized cloud - like graphics,They weave through the crowd.,Stylized cloud - like graphics,gen,play squashed between two trees by the illuminated smoke.,rises into the sky.,pass the black spacecraft and soars.,"appear in the screen, under someone.",1
1,lsmdc1061_Harry_Potter_and_the_deathly_hallows_Disk_Two-100176,4770,"Then, they both look up at the Willow tree above them. The wind rustles the branches and the leaves","Then, they both look up at the Willow tree above them.",The wind rustles the branches and the leaves,gold,lashes across the crevice of the tiger's features.,struggle to brace themselves.,fall flying like tiny birds.,rip out the last roots.,2
2,lsmdc3087_WE_BOUGHT_A_ZOO-41265,12240,"As the visitor turns, we recognize Ferris' features from the photo on the dartboard. Wearing a suit beneath a rugged parka, he","As the visitor turns, we recognize Ferris' features from the photo on the dartboard.","Wearing a suit beneath a rugged parka, he",gold,approaches with a cocky disdainful smirk.,faces several record intro texting.,follows his staff outside.,rises to his feet and hurries to the door.,0
3,lsmdc3056_PUBLIC_ENEMIES-2753,19192,"They each pull out a gun and using the spools to conceal them, pass them to the young inmate and man sitting next to him. They","They each pull out a gun and using the spools to conceal them, pass them to the young inmate and man sitting next to him.",They,gold,stealthily tuck the weapons in their pants.,carry their heads and carry him down the back stilts onto a catwalk.,continue to crane the driver down.,shows the other way to work out.,0
4,anetv_GPR8rtfZr4A,10131,A red handball is rolled to a kicker during a kickball match who kicks it into the outfield for a run. Another man in tshirt and black sweats,A red handball is rolled to a kicker during a kickball match who kicks it into the outfield for a run.,Another man in tshirt and black sweats,gold,runs throughout the room to the finish bar.,kicks a ball and runs around the bases trying to avoid being tagged with the ball.,does another cheer that runs back to the table and puts purple arms on the ground speaking to the camera.,starts kicking and punching and throwing it across the court.,1
5,lsmdc0030_The_Hustler-64567,8354,"The balls roll down the table, bank off the far shoulder, and slowly return toward the two players. Someone 'ball","The balls roll down the table, bank off the far shoulder, and slowly return toward the two players.",Someone 'ball,gold,rips into men's skin.,emerges from time to far right and alongside the man's.,hits the closer shoulder.,drops less than a short.,2
6,anetv_4bUxtqX_oxM,8619,There is a row of people riding camels behind the woman. There,There is a row of people riding camels behind the woman.,There,gold,see the darts at a rodeo.,is dribbling the ball to lantern who is playing with the large ball.,are busy behind them.,is one person in front of the woman riding a camel.,3
7,anetv_2icoQWmbocU,5224,They ride horses and bow as the man appears to fight the leader and save a woman tied to a stake. He,They ride horses and bow as the man appears to fight the leader and save a woman tied to a stake.,He,gen,takes him away passionately.,"chases after it, ripping it down.",stretches up the show above his finger.,shoots up as he tries to fight away.,3
8,lsmdc1026_Legion-5666,9281,Someone takes the machine pistol and picks up an assault rifle. People,Someone takes the machine pistol and picks up an assault rifle.,People,gold,flip into the ground.,jab at each other.,pulls the planks away from the door and open it.,shines a torch within a metal shield.,2
9,lsmdc0025_THE_LORD_OF_THE_RINGS_THE_RETURN_OF_THE_KING-60961,7324,People move through the camp. Someone's eyes,People move through the camp.,Someone's eyes,gold,follow the tile in the puddles as he sits between t

In [12]:
def show_one(example):
    print(f"Context: {example['sent1']}")
    print(f"  A - {example['sent2']} {example['ending0']}")
    print(f"  B - {example['sent2']} {example['ending1']}")
    print(f"  C - {example['sent2']} {example['ending2']}")
    print(f"  D - {example['sent2']} {example['ending3']}")
    print(f"\nGround truth: option {['A', 'B', 'C', 'D'][example['label']]}")

In [13]:
show_one(datasets["train"][0])

Context: Members of the procession walk down the street holding small horn brass instruments.
  A - A drum line passes by walking down the street playing their instruments.
  B - A drum line has heard approaching them.
  C - A drum line arrives and they're outside dancing and asleep.
  D - A drum line turns the lead singer watches the performance.

Ground truth: option A


In [14]:
show_one(datasets["train"][6])

Context: A person retrieves an instrument from a closet.
  A - The man examines the instrument in his hand.
  B - The man stops playing the drums and waves over the other boys.
  C - The man lights the cigarette and sticks his head in.
  D - The man drags off the vacuum.

Ground truth: option A


##Preprocessing

In [15]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

In [16]:
tokenizer("Hello, this one sentence!", "And this sentence goes with it.")

{'input_ids': [101, 7592, 1010, 2023, 2028, 6251, 999, 102, 1998, 2023, 6251, 3632, 2007, 2009, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [17]:
ending_names = ["ending0", "ending1", "ending2", "ending3"]

def preprocess_function(examples):
    # Repeat each first sentence four times to go with the four possibilities of second sentences.
    first_sentences = [[context] * 4 for context in examples["sent1"]]
    # Grab all second sentences possible for each context.
    question_headers = examples["sent2"]
    second_sentences = [[f"{header} {examples[end][i]}" for end in ending_names] for i, header in enumerate(question_headers)]
    
    # Flatten everything
    first_sentences = sum(first_sentences, [])
    second_sentences = sum(second_sentences, [])
    
    # Tokenize
    tokenized_examples = tokenizer(first_sentences, second_sentences, truncation=True)
    # Un-flatten
    return {k: [v[i:i+4] for i in range(0, len(v), 4)] for k, v in tokenized_examples.items()}

In [18]:
examples = datasets["train"][:5]
features = preprocess_function(examples)
print(len(features["input_ids"]), len(features["input_ids"][0]),\
      [len(x) for x in features["input_ids"][0]])

5 4 [30, 25, 30, 28]


In [19]:
idx = 3
[tokenizer.decode(features["input_ids"][idx][i]) for i in range(4)]

['[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession are playing ping pong and celebrating one left each in quick. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession wait slowly towards the cadets. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions. [SEP]',
 '[CLS] a drum line passes by walking down the street playing their instruments. [SEP] members of the procession play and go back and forth hitting the drums while the audience claps for them. [SEP]']

In [20]:
show_one(datasets["train"][3])

Context: A drum line passes by walking down the street playing their instruments.
  A - Members of the procession are playing ping pong and celebrating one left each in quick.
  B - Members of the procession wait slowly towards the cadets.
  C - Members of the procession makes a square call and ends by jumping down into snowy streets where fans begin to take their positions.
  D - Members of the procession play and go back and forth hitting the drums while the audience claps for them.

Ground truth: option D


In [21]:
encoded_datasets = datasets.map(preprocess_function, batched=True)

  0%|          | 0/74 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

##Fine-tuning

In [22]:
model = AutoModelForMultipleChoice.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [23]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-swag",
    evaluation_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01
)


In [24]:
@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch

In [25]:
accepted_keys = ["input_ids", "attention_mask", "label"]
features = [{k: v for k, v in encoded_datasets["train"][i].items() if k in accepted_keys} for i in range(10)]
batch = DataCollatorForMultipleChoice(tokenizer)(features)

In [26]:
[tokenizer.decode(batch["input_ids"][8][i].tolist()) for i in range(4)]

['[CLS] someone walks over to the radio. [SEP] someone hands her another phone. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone takes the drink, then holds it. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone looks off then looks at someone. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]',
 '[CLS] someone walks over to the radio. [SEP] someone stares blearily down at the floor. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]']

In [27]:
show_one(datasets["train"][8])

Context: Someone walks over to the radio.
  A - Someone hands her another phone.
  B - Someone takes the drink, then holds it.
  C - Someone looks off then looks at someone.
  D - Someone stares blearily down at the floor.

Ground truth: option D


In [28]:
def compute_metrics(eval_predictions):
    predictions, label_ids = eval_predictions
    preds = np.argmax(predictions, axis=1)
    return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

In [29]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_datasets["train"],
    eval_dataset=encoded_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer),
    compute_metrics=compute_metrics,
)

In [30]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForMultipleChoice.forward` and have been ignored: video-id, sent1, ending3, ending0, startphrase, fold-ind, ending2, gold-source, sent2, ending1.
***** Running training *****
  Num examples = 73546
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 13791


Epoch,Training Loss,Validation Loss,Accuracy
1,0.761700,0.607853,0.765020
2,0.390700,0.613603,0.785364
3,0.140500,1.045711,0.788413


Saving model checkpoint to bert-base-uncased-finetuned-swag/checkpoint-500
Configuration saved in bert-base-uncased-finetuned-swag/checkpoint-500/config.json
Model weights saved in bert-base-uncased-finetuned-swag/checkpoint-500/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-swag/checkpoint-500/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-swag/checkpoint-500/special_tokens_map.json
Saving model checkpoint to bert-base-uncased-finetuned-swag/checkpoint-1000
Configuration saved in bert-base-uncased-finetuned-swag/checkpoint-1000/config.json
Model weights saved in bert-base-uncased-finetuned-swag/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in bert-base-uncased-finetuned-swag/checkpoint-1000/tokenizer_config.json
Special tokens file saved in bert-base-uncased-finetuned-swag/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to bert-base-uncased-finetuned-swag/checkpoint-1500
Configuration saved i

TrainOutput(global_step=13791, training_loss=0.457729687848747, metrics={'train_runtime': 5285.6063, 'train_samples_per_second': 41.743, 'train_steps_per_second': 2.609, 'total_flos': 2.466167112293952e+16, 'train_loss': 0.457729687848747, 'epoch': 3.0})